In [ ]:
"""
1) Apply Resampling method to your regression and classification tasks (40%).
    a) Cross Validation
    b) Bootstrap
2) Apply model selection method for your regression and classification tasks (40%).
    a) Subset Selection (Cp, AIC, BIC, and Adjusted R2)
    b) Shrinkage (Ridge and Lasso Regression)
    c) Dimension Reduction (PCA)
3) Write a report (20%).
"""

In [9]:
import pandas as pd

In [6]:
df_reg = pd.read_csv('/Users/eljan/Documents/ML-Course-ADA/project-1/binaaz_cleaned.csv')
df_class = pd.read_csv("/Users/eljan/Documents/ML-Course-ADA/project-2/credit_score.csv")
df_class.drop(columns=["Unnamed: 0"], inplace=True)
df_class = df_class[df_class['Credit_Score'] != 1]
df_class['Credit_Score'] = df_class['Credit_Score'].map({2: 0, 0: 1})

In [7]:
df_reg.head()

,price,poster_type,Mərtəbə,Sahə,Otaq sayı,Kupça,İpoteka,distance_city_center,rayon_Abşeron,rayon_Binəqədi,...,metro_Nəsimi,metro_Qara Qarayev,metro_Sahil,metro_Ulduz,metro_Xalqlar Dostluğu,metro_İnşaatçılar,metro_İçəri Şəhər,metro_Şah İsmayıl Xətai,metro_Əhmədli,seher_baki
0,300000,0,0.294118,135.0,3,1,0,1.645502,0,0,...,0,0,0,0,0,0,0,0,0,1
1,89900,0,0.666667,65.0,2,1,0,4.888944,0,0,...,0,0,0,0,0,0,0,0,0,1
2,142000,0,0.647059,115.0,3,1,0,8.291625,0,0,...,0,0,0,0,0,0,0,0,0,1
3,235000,0,1.000000,192.0,4,1,0,6.159160,0,0,...,0,0,0,0,0,0,0,0,0,1
4,235000,0,0.750000,107.0,2,0,0,5.461605,0,0,...,0,0,0,0,0,0,0,0,0,1


In [8]:
df_class.head()

,Month,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,2,23.000000,13,19114.12,4194.170850,3.0,4.0,3.0,4.0,128,...,1,809.98,31.944960,0,1,49.574949,118.280222,3,284.629162,0
1,6,34.429817,13,19114.12,4194.170850,3.0,4.0,3.0,4.0,128,...,1,809.98,28.609352,267,1,49.574949,81.699521,4,331.209863,0
2,0,23.000000,13,19114.12,4194.170850,3.0,4.0,3.0,4.0,128,...,1,809.98,31.377862,268,1,49.574949,199.458074,5,223.451310,0
3,7,23.000000,13,19114.12,1824.843333,3.0,4.0,3.0,4.0,128,...,1,809.98,24.797347,269,1,49.574949,41.420153,1,341.489231,0
4,5,23.000000,13,19114.12,4194.170850,3.0,4.0,3.0,4.0,128,...,1,809.98,27.262259,270,1,49.574949,62.430172,6,340.479212,0


# Apply Resampling method

# Baseline

## Regression

In [10]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures

# Split data
X = df_reg.drop('price', axis=1)
y = df_reg['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Linear Regression
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
y_pred_lin = lin_reg.predict(X_test)
rss_lin = np.sum((y_pred_lin - y_test)**2)
rmse_lin = np.sqrt(mean_squared_error(y_test, y_pred_lin))

# Polynomial Regression
poly = PolynomialFeatures(degree=2)
X_poly_train = poly.fit_transform(X_train)
X_poly_test = poly.transform(X_test)
poly_reg = LinearRegression()
poly_reg.fit(X_poly_train, y_train)
y_pred_poly = poly_reg.predict(X_poly_test)
rss_poly = np.sum((y_pred_poly - y_test)**2)
rmse_poly = np.sqrt(mean_squared_error(y_test, y_pred_poly))

print("Linear Regression - RSS:", rss_lin, "RMSE:", rmse_lin)
print("Polynomial Regression - RSS:", rss_poly, "RMSE:", rmse_poly)


Linear Regression - RSS: 7045528535081.69 RMSE: 36791.409156484144
Polynomial Regression - RSS: 7.91459220351031e+22 RMSE: 3899454924.7188563


## Classification

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve, auc

def print_evaluation(y_true, y_pred):
    print(confusion_matrix(y_true, y_pred))
    print(classification_report(y_true, y_pred))
    fpr, tpr, thresholds = roc_curve(y_true, y_pred, pos_label=1)
    print("AUC:", auc(fpr, tpr))

# Split data
X = df_class.drop('Credit_Score', axis=1)
y = df_class['Credit_Score']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Logistic Regression
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)
y_pred_log = log_reg.predict(X_test)
print_evaluation(y_test, y_pred_log)

# LDA
lda = LinearDiscriminantAnalysis()
lda.fit(X_train, y_train)
y_pred_lda = lda.predict(X_test)
print_evaluation(y_test, y_pred_lda)

# QDA
qda = QuadraticDiscriminantAnalysis()
qda.fit(X_train, y_train)
y_pred_qda = qda.predict(X_test)
print_evaluation(y_test, y_pred_qda)

# Naive Bayes
nb = GaussianNB()
nb.fit(X_train, y_train)
y_pred_nb = nb.predict(X_test)
print_evaluation(y_test, y_pred_nb)




/Users/eljan/anaconda3/envs/polygraf/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[[1922  953]
 [ 761 3831]]
              precision    recall  f1-score   support

           0       0.72      0.67      0.69      2875
           1       0.80      0.83      0.82      4592

    accuracy                           0.77      7467
   macro avg       0.76      0.75      0.75      7467
weighted avg       0.77      0.77      0.77      7467

AUC: 0.7513993713073777
[[2534  341]
 [ 834 3758]]
              precision    recall  f1-score   support

           0       0.75      0.88      0.81      2875
           1       0.92      0.82      0.86      4592

    accuracy                           0.84      7467
   macro avg       0.83      0.85      0.84      7467
weighted avg       0.85      0.84      0.84      7467

AUC: 0.8498855476442962
[[2545  330]
 [ 830 3762]]
              precision    recall  f1-score   support

           0       0.75      0.89      0.81      2875
           1       0.92      0.82      0.87      4592

    accuracy                           0.84      7467

## Regression

In [24]:
from sklearn.model_selection import cross_val_score

# Linear Regression with Cross-Validation
lin_reg_cv_scores = cross_val_score(lin_reg, X, y, cv=10, scoring='neg_mean_squared_error')
print("Linear Regression - CV RMSE:", np.sqrt(-lin_reg_cv_scores.mean()))

# Polynomial Features Transformation
X_poly = poly.fit_transform(X)

# Polynomial Regression with Cross-Validation
poly_reg_cv_scores = cross_val_score(poly_reg, X_poly, y, cv=10, scoring='neg_mean_squared_error')
print("Polynomial Regression - CV RMSE:", np.sqrt(-poly_reg_cv_scores.mean()))


Linear Regression - CV RMSE: 36908.9668594989
Polynomial Regression - CV RMSE: 22995436493.26417


In [25]:
from sklearn.utils import resample

def bootstrap_regression(model, X, y, n_bootstraps=100):
    mse_scores = []
    
    for _ in range(n_bootstraps):
        X_boot, y_boot = resample(X, y)
        model.fit(X_boot, y_boot)
        y_pred = model.predict(X)
        mse_scores.append(mean_squared_error(y, y_pred))
    
    print(f"{model.__class__.__name__} - Bootstrap RMSE:", np.sqrt(np.mean(mse_scores)))

# Linear Regression with Bootstrap
bootstrap_regression(lin_reg, X, y)

# Polynomial Regression with Bootstrap
bootstrap_regression(poly_reg, X_poly, y)


LinearRegression - Bootstrap RMSE: 36835.70944678861
LinearRegression - Bootstrap RMSE: 11645958360.905136


## Classification

In [19]:
from sklearn.model_selection import cross_val_score

X = df_class.drop('Credit_Score', axis=1)
y = df_class['Credit_Score']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Helper function to calculate mean AUC
def cv_auc(model, X, y, cv=10):
    scores = cross_val_score(model, X, y, cv=cv, scoring='roc_auc')
    print(f"{model.__class__.__name__} - CV AUC:", scores.mean())

# Logistic Regression with Cross-Validation
cv_auc(log_reg, X, y)

# LDA with Cross-Validation
cv_auc(lda, X, y)

# QDA with Cross-Validation
cv_auc(qda, X, y)

# Naive Bayes with Cross-Validation
cv_auc(nb, X, y)


/Users/eljan/anaconda3/envs/polygraf/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/eljan/anaconda3/envs/polygraf/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/l

LogisticRegression - CV AUC: 0.8565264703508741
LinearDiscriminantAnalysis - CV AUC: 0.9070482896959419
QuadraticDiscriminantAnalysis - CV AUC: 0.8944014680113561
GaussianNB - CV AUC: 0.8541506741600058


In [20]:
from sklearn.metrics import roc_auc_score

def bootstrap_classification(model, X, y, n_bootstraps=100):
    auc_scores = []
    
    for _ in range(n_bootstraps):
        X_boot, y_boot = resample(X, y)
        model.fit(X_boot, y_boot)
        y_pred = model.predict(X)
        auc_scores.append(roc_auc_score(y, y_pred))
    
    print(f"{model.__class__.__name__} - Bootstrap AUC:", np.mean(auc_scores))

# Logistic Regression with Bootstrap
bootstrap_classification(log_reg, X, y)

# LDA with Bootstrap
bootstrap_classification(lda, X, y)

# QDA with Bootstrap
bootstrap_classification(qda, X, y)

# Naive Bayes with Bootstrap
bootstrap_classification(nb, X, y)


/Users/eljan/anaconda3/envs/polygraf/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/eljan/anaconda3/envs/polygraf/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/l

LogisticRegression - Bootstrap AUC: 0.7628732347468217
LinearDiscriminantAnalysis - Bootstrap AUC: 0.8489355443923593
QuadraticDiscriminantAnalysis - Bootstrap AUC: 0.8485714736889372
GaussianNB - Bootstrap AUC: 0.8053207033688902


# Apply model selection method

## Regression

### AIC and BIC subset selection

In [29]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split

def stepwise_selection(X, y, initial_list=[], threshold_in=0.01, threshold_out=0.05, criterion='AIC'):
    """ Perform a forward-backward feature selection
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        criterion - 'AIC' or 'BIC' as the criterion to select the best model
    Returns: list of selected features 
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns) - set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included + [new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed=True
            while True:
                # backward step
                model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
                # use all coefs except intercept
                pvalues = model.pvalues.iloc[1:]
                worst_pval = pvalues.max()  # null if pvalues is empty
                if worst_pval > threshold_out:
                    changed=True
                    worst_feature = pvalues.idxmax()
                    included.remove(worst_feature)
                else:
                    break
        if not changed:
            break
    model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
    if criterion == 'AIC':
        criterion_value = model.aic
    elif criterion == 'BIC':
        criterion_value = model.bic
    print('Resulting features:', included)
    print(f'Model {criterion}: {criterion_value}')
    return included

# Data preparation
X = df_reg.drop('price', axis=1)
y = df_reg['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Applying stepwise selection using AIC
aic_selected_features = stepwise_selection(X_train, y_train, criterion='AIC')
bic_selected_features = stepwise_selection(X_train, y_train, criterion='AIC')


Resulting features: ['seher_baki', 'Sahə', 'distance_city_center', 'metro_Elmlər Akademiyası', 'Kupça', 'metro_İçəri Şəhər', 'metro_Şah İsmayıl Xətai', 'metro_Nizami', 'metro_28 May', 'Otaq sayı', 'rayon_Binəqədi', 'metro_Sahil', 'metro_İnşaatçılar', 'İpoteka', 'metro_Bakmil', 'rayon_Sabunçu', 'metro_Azadlıq Prospekti', 'Mərtəbə', 'rayon_Abşeron']
Model AIC: 497112.04402335285
Resulting features: ['seher_baki', 'Sahə', 'distance_city_center', 'metro_Elmlər Akademiyası', 'Kupça', 'metro_İçəri Şəhər', 'metro_Şah İsmayıl Xətai', 'metro_Nizami', 'metro_28 May', 'Otaq sayı', 'rayon_Binəqədi', 'metro_Sahil', 'metro_İnşaatçılar', 'İpoteka', 'metro_Bakmil', 'rayon_Sabunçu', 'metro_Azadlıq Prospekti', 'Mərtəbə', 'rayon_Abşeron']
Model AIC: 497112.04402335285


In [32]:
X.shape, len(aic_selected_features), len(bic_selected_features)

((26025, 43), 19, 19)

### Adjusted R2 

In [36]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split

def forward_selection_by_adj_r_squared(X, y, max_features=5):
    """Perform a forward feature selection based on Adjusted R-squared from statsmodels.api.OLS"""
    initial_features = X.columns.tolist()
    best_features = []
    best_adj_r_squared = -np.inf  # Start with the worst case scenario
    
    while len(best_features) < max_features:
        current_best_adj_r_squared = -np.inf
        candidate_feature = None
        
        for new_column in list(set(initial_features) - set(best_features)):
            model = sm.OLS(y, sm.add_constant(X[best_features + [new_column]])).fit()
            adj_r_squared = model.rsquared_adj
            if adj_r_squared > current_best_adj_r_squared:
                current_best_adj_r_squared = adj_r_squared
                candidate_feature = new_column

        # Check if the new candidate feature improves the model based on Adjusted R-squared
        if current_best_adj_r_squared > best_adj_r_squared:
            best_features.append(candidate_feature)
            best_adj_r_squared = current_best_adj_r_squared
        else:
            break  # Stop if no improvement

    # Fit the final model with the selected features
    final_model = sm.OLS(y, sm.add_constant(X[best_features])).fit()
    return best_features, final_model

# Data preparation
X = df_reg.drop('price', axis=1)
y = df_reg['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply forward selection focused on Adjusted R-squared
selected_features, selected_model = forward_selection_by_adj_r_squared(X_train, y_train, max_features=5)

print("Selected Features:", selected_features)
print(selected_model.summary())


Selected Features: ['Sahə', 'distance_city_center', 'metro_Elmlər Akademiyası', 'Kupça', 'metro_İçəri Şəhər']
                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.698
Model:                            OLS   Adj. R-squared:                  0.698
Method:                 Least Squares   F-statistic:                     9624.
Date:                Tue, 16 Apr 2024   Prob (F-statistic):               0.00
Time:                        00:41:26   Log-Likelihood:            -2.5026e+05
No. Observations:               20820   AIC:                         5.005e+05
Df Residuals:                   20814   BIC:                         5.006e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
-------

### Mallows' Cp

In [37]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

def forward_selection_cp(X, y, mse_full, max_features=5):
    """Perform a forward feature selection based on minimizing Mallows' Cp."""
    initial_features = X.columns.tolist()
    best_features = []
    best_cp = np.inf
    
    while len(best_features) < max_features:
        current_best_cp = np.inf
        candidate_feature = None
        
        for new_column in list(set(initial_features) - set(best_features)):
            temp_features = best_features + [new_column]
            model = sm.OLS(y, sm.add_constant(X[temp_features])).fit()
            mse_model = mean_squared_error(y, model.predict(sm.add_constant(X[temp_features])))
            n = len(y)
            p = len(temp_features)
            cp = (mse_model / mse_full) * n - (n - 2 * p)
            
            if cp < current_best_cp:
                current_best_cp = cp
                candidate_feature = new_column

        # Check if the new candidate feature improves the Cp value
        if current_best_cp < best_cp:
            best_features.append(candidate_feature)
            best_cp = current_best_cp
        else:
            break  # Stop if no improvement in Cp

    # Fit the final model with the selected features
    final_model = sm.OLS(y, sm.add_constant(X[best_features])).fit()
    return best_features, final_model, best_cp

# Data preparation
X = df_reg.drop('price', axis=1)
y = df_reg['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit the full model to get the MSE of the full model
full_model = sm.OLS(y_train, sm.add_constant(X_train)).fit()
mse_full = mean_squared_error(y_train, full_model.predict(sm.add_constant(X_train)))

# Apply forward selection focused on Mallows' Cp
selected_features, selected_model, selected_cp = forward_selection_cp(X_train, y_train, mse_full, max_features=5)

print("Selected Features:", selected_features)
print("Mallows' Cp:", selected_cp)
print(selected_model.summary())


Selected Features: ['Sahə', 'distance_city_center', 'metro_Elmlər Akademiyası', 'Kupça', 'metro_İçəri Şəhər']
Mallows' Cp: 3976.805512615272
                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.698
Model:                            OLS   Adj. R-squared:                  0.698
Method:                 Least Squares   F-statistic:                     9624.
Date:                Tue, 16 Apr 2024   Prob (F-statistic):               0.00
Time:                        00:44:38   Log-Likelihood:            -2.5026e+05
No. Observations:               20820   AIC:                         5.005e+05
Df Residuals:                   20814   BIC:                         5.006e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t| 

## Classification

## Regression with Ridge and Lasso Shrinkage

In [40]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import mean_squared_error

# Assuming 'df_reg' is your regression DataFrame

# Split data
X = df_reg.drop('price', axis=1)
y = df_reg['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Ridge Regression
ridge_reg = Ridge(alpha=1.0)  # alpha is the regularization strength
ridge_reg.fit(X_train, y_train)
y_pred_ridge = ridge_reg.predict(X_test)
rss_ridge = np.sum((y_pred_ridge - y_test)**2)
rmse_ridge = np.sqrt(mean_squared_error(y_test, y_pred_ridge))

# Lasso Regression
lasso_reg = Lasso(alpha=0.1)  # alpha can be tuned
lasso_reg.fit(X_train, y_train)
y_pred_lasso = lasso_reg.predict(X_test)
rss_lasso = np.sum((y_pred_lasso - y_test)**2)
rmse_lasso = np.sqrt(mean_squared_error(y_test, y_pred_lasso))

print("Ridge Regression - RSS:", rss_ridge, "RMSE:", rmse_ridge)
print("Lasso Regression - RSS:", rss_lasso, "RMSE:", rmse_lasso)


Ridge Regression - RSS: 7039801204478.572 RMSE: 36776.45219456246
Lasso Regression - RSS: 7045031728183.132 RMSE: 36790.11198282189


/Users/eljan/anaconda3/envs/polygraf/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.737e+12, tolerance: 1.113e+10
  model = cd_fast.enet_coordinate_descent(


## Regression with PCA:

In [41]:
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression

# PCA with Linear Regression for regression task
pca = PCA(n_components=0.95)  # retains 95% of variance
pca_reg = make_pipeline(pca, LinearRegression())
pca_reg.fit(X_train, y_train)
y_pred_pca_reg = pca_reg.predict(X_test)
rss_pca_reg = np.sum((y_pred_pca_reg - y_test)**2)
rmse_pca_reg = np.sqrt(mean_squared_error(y_test, y_pred_pca_reg))

print("PCA with Linear Regression - RSS:", rss_pca_reg, "RMSE:", rmse_pca_reg)


PCA with Linear Regression - RSS: 10758897384774.242 RMSE: 45464.61478824245


In [52]:
from sklearn.linear_model import LogisticRegression


X = df_class.drop('Credit_Score', axis=1)
y = df_class['Credit_Score']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# PCA with Logistic Regression for classification task
pca_log_reg = make_pipeline(PCA(n_components=0.8), LogisticRegression(class_weight='balanced'))
pca_log_reg.fit(X_train, y_train)
y_pred_pca_log = pca_log_reg.predict(X_test)
print_evaluation(y_test, y_pred_pca_log)


[[  94 2781]
 [  57 4535]]
              precision    recall  f1-score   support

           0       0.62      0.03      0.06      2875
           1       0.62      0.99      0.76      4592

    accuracy                           0.62      7467
   macro avg       0.62      0.51      0.41      7467
weighted avg       0.62      0.62      0.49      7467

AUC: 0.5101413800939252


In [50]:
# apply Logistic Regression
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)
y_pred_log = log_reg.predict(X_test)
print_evaluation(y_test, y_pred_log)

[[1922  953]
 [ 761 3831]]
              precision    recall  f1-score   support

           0       0.72      0.67      0.69      2875
           1       0.80      0.83      0.82      4592

    accuracy                           0.77      7467
   macro avg       0.76      0.75      0.75      7467
weighted avg       0.77      0.77      0.77      7467

AUC: 0.7513993713073777


/Users/eljan/anaconda3/envs/polygraf/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
